In [1]:
import json
import os
import pandas as pd
import numpy as np

In [2]:
vocabs_data = pd.read_csv('../data/csv/vocabs_20220612.csv')
vocabs_data.head()

,日期,單字,詞性,註解,詞性.1,註解.1,詞性.2,註解.2
0,12/06/2022,delegate,動,委託,NaN,NaN,NaN,NaN
1,12/06/2022,broaden,動,擴大；擴增,NaN,NaN,NaN,NaN
2,12/06/2022,supplement,動,補充,NaN,NaN,NaN,NaN
3,12/06/2022,emerge,動,浮現,NaN,NaN,NaN,NaN
4,12/06/2022,pardon,動,諒解；寛恕,NaN,NaN,NaN,NaN


In [3]:
def build_vocab_meta(vocab, vocab_data, col_speech, col_mean):    
    if vocab != vocab_data['單字']:
        print("[%s] given vocab and data not match !!!!!" % vocab)
        return 0
    
    meta = {}
    meta['mean'] = {}
    for speech, mean in zip(col_speech, col_mean):
        if pd.isna(vocab_data[speech]) & pd.isna(vocab_data[mean]):
            continue
        elif pd.isna(vocab_data[speech]) + pd.isna(vocab_data[mean]) == 1:
            print("[%s] invalid speech or mean (one of them is NaN)")
        meta['mean'][vocab_data[speech]] = vocab_data[mean]
    meta['date'] = vocab_data['日期']
    meta['proficiency'] = 1
    return meta

In [4]:
vocabs = vocabs_data['單字'].dropna(axis=0).tolist()
col_speech = []
col_mean = []
max_col_speech = 0
max_col_mean = 0

for i, col in enumerate(vocabs_data.columns):
    if '詞性' in col:
        col_speech.append(col)
        max_col_speech += 1
    if '註解' in col:
        col_mean.append(col)
        max_col_mean += 1
vocabs_meta = {}

In [5]:
### Test a data
# i = 48
# vocabs_meta[vocabs[i]] = build_vocab_meta(vocabs[i], vocabs_data.iloc[i], col_speech, col_mean)

### Start to Build Meta of Vocabs
vocabs_count = len(vocabs)
repeat_vocabs_count = 0
print("Current size of vocabs : %d" % vocabs_count)
for i in range(vocabs_count):
    if vocabs[i] in vocabs_meta.keys():
        print("[%s] is already exist" % vocabs[i])
        repeat_vocabs_count += 1
        continue
    vocabs_meta[vocabs[i]] = build_vocab_meta(vocabs[i], vocabs_data.iloc[i], col_speech, col_mean)
print("New size of vocabs : %d" % (vocabs_count-repeat_vocabs_count))

Current size of vocabs : 258
[delegate] is already exist
[essential] is already exist
[absence] is already exist
[react] is already exist
[senator] is already exist
[incentive] is already exist
[incline] is already exist
[attendant] is already exist
[incentive] is already exist
[react] is already exist
[grant] is already exist
New size of vocabs : 247


In [ ]:
save = None
_, ext = os.path.splitext(save)
if ext == '.json':
    corpus = save
elif not ext:
    corpus = os.path.join(save, "vocabs.json")
else:
    raise ValueError("the config --save must be directory or file with ext '.json'")
with open(corpus, 'w') as f:
    json.dump(vocabs_meta, f, indent=4)